In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [24]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
import pickle

In [25]:
data=pd.read_csv("Material Compressive Strength Experimental Data.csv")

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6139 entries, 0 to 6138
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Material Quantity (gm)      6030 non-null   float64
 1   Additive Catalyst (gm)      6030 non-null   float64
 2   Ash Component (gm)          6030 non-null   float64
 3   Water Mix (ml)              6030 non-null   float64
 4   Plasticizer (gm)            6030 non-null   float64
 5   Moderate Aggregator         6030 non-null   float64
 6   Refined Aggregator          6030 non-null   float64
 7   Formulation Duration (hrs)  6030 non-null   float64
 8   Compression Strength MPa    6139 non-null   float64
dtypes: float64(9)
memory usage: 431.8 KB


In [27]:
data.describe()

,Material Quantity (gm),Additive Catalyst (gm),Ash Component (gm),Water Mix (ml),Plasticizer (gm),Moderate Aggregator,Refined Aggregator,Formulation Duration (hrs),Compression Strength MPa
count,6030.000000,6030.000000,6030.000000,6030.000000,6030.000000,6030.000000,6030.000000,6030.000000,6139.000000
mean,383.642297,196.699846,111.856252,224.296955,17.651085,998.669332,811.832398,174.408504,56.851430
std,149.994316,133.329220,74.241117,41.545751,11.687965,97.732677,112.813539,112.415173,16.124932
min,124.440000,0.000000,0.000000,148.600000,0.000000,821.540000,609.230000,16.250000,2.610000
25%,256.030000,78.210000,44.582500,190.387500,7.922500,918.437500,717.447500,70.300000,47.085000
50%,377.405000,192.320000,115.250000,225.700000,16.345000,997.985000,810.260000,163.105000,59.790000
75%,511.522500,307.650000,174.257500,257.447500,27.667500,1079.827500,905.857500,272.602500,69.845000
max,658.800000,438.470000,244.120000,301.340000,39.280000,1174.360000,1018.050000,380.250000,92.510000


In [28]:
li=list(data.columns)
data1=data.copy()
for i in li:
    data1[i]=data[i].fillna(data[i].mean())
#     print(data1[i].isnull().sum())
# we will use data1 dateset in our model

In [29]:
inp=data1.drop("Compression Strength MPa",axis=1)
out=data1[["Compression Strength MPa"]]


In [30]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(inp,out,test_size=0.2,random_state=797)

In [31]:
print(f"size of train data {x_train.shape},{y_train.shape}")
print(f"size of test data {x_test.shape},{y_test.shape}")

size of train data (4911, 8),(4911, 1)
size of test data (1228, 8),(1228, 1)


In [32]:
Rf=RandomForestRegressor()

In [40]:
pipeline=Pipeline(steps=[
    ("standardscaler",StandardScaler()),
    ("minmaxscaler",MinMaxScaler()),
    ("model",Rf)
] )

In [41]:
pipeline.fit(x_train,y_train)

c:\Users\tanushree\Desktop\ML_project\ML_deployment\env\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('minmaxscaler', MinMaxScaler()),
                ('model', RandomForestRegressor())])

In [42]:
y_pred=pipeline.predict(x_test)
y_pred

array([56.8852, 61.0238, 56.8168, ..., 55.1063, 52.807 , 63.7744])

In [43]:
score1=r2_score(y_test,y_pred)
score1

0.4346926037496305

In [44]:
print("mean squared error is ",mean_squared_error(y_test,y_pred))
print("mean absolute error is ",mean_absolute_error(y_test,y_pred))


mean squared error is  148.06325535123585
mean absolute error is  9.640600557483685


In [52]:
with open("pickle_cement.pkl","wb") as file:
     pickle.dump(pipeline,file)
with open("pickle_cement.pkl","rb") as file:
     Rf_model = pickle.load(file)

In [17]:
# pickle.dump(pipline,open("pickle_cement.pkl","wb"))
# Rf_model=pickle.load(open("pickle_cement.pkl","rb"))

In [50]:
list_=[441.63,437.58,92.38,180.12,10.19,913.68,1018.05,372.63]
list_=np.array(list_).reshape(1,-1)
print(list_.shape)
list_

(1, 8)


array([[ 441.63,  437.58,   92.38,  180.12,   10.19,  913.68, 1018.05,
         372.63]])

In [51]:
 Rf_model.predict(list_)


c:\Users\tanushree\Desktop\ML_project\ML_deployment\env\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([56.4423])